# 0. Set up

In [1]:
## Import packages
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
## Set paths
user = os.getenv('USERNAME')
shared_workspace = '/home/mdawkins/modelling_club'
user_dir = os.path.join(shared_workspace, user)
data_dir = os.path.join(shared_workspace, 'raw_data')
data_output_dir = os.path.join(shared_workspace, 'raw_data_lfs/engineered/POS_CASH_balance/')

# 1. Import data

In [3]:
home_loan_train = pd.read_csv(data_dir + '/raw/application_train.csv',index_col='SK_ID_CURR')
train_response = home_loan_train['TARGET']

home_loan_test = pd.read_csv(data_dir + '/raw/application_test_noTarget.csv',index_col='SK_ID_CURR')
home_loan_test.loc[:, 'TARGET'] = None

pos = pd.read_csv(data_dir + '/raw/POS_CASH_balance.csv')
pos

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
...,...,...,...,...,...,...,...,...
8543370,2448283,226558,-20,6.0,0.0,Active,843,0
8543371,1717234,141565,-19,12.0,0.0,Active,602,0
8543372,1283126,315695,-21,10.0,0.0,Active,609,0
8543373,1082516,450255,-22,12.0,0.0,Active,614,0


In [15]:
test = pos[pos['SK_DPD']>0]
test[test['MONTHS_BALANCE']>-10]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
6375,1121698,417906,-5,12.0,3.0,Active,8,8
7831,1467136,125771,-6,12.0,2.0,Active,2,2
8047,2675685,224293,-6,36.0,19.0,Active,4,4
8624,1754696,128260,-4,12.0,2.0,Active,3,3
8952,2675188,350201,-2,24.0,7.0,Active,14,0
...,...,...,...,...,...,...,...,...
8542795,2439827,116933,-4,16.0,0.0,Active,1012,0
8542796,1334004,157645,-4,6.0,0.0,Active,1264,0
8542800,2210180,392923,-8,6.0,0.0,Active,1021,0
8542804,2780514,199205,-9,6.0,0.0,Active,968,0


In [16]:
pos.loc[pos['SK_ID_PREV']==2675188].sort_values('MONTHS_BALANCE')

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
3024860,2675188,350201,-19,24.0,24.0,Active,0,0
1079795,2675188,350201,-18,24.0,23.0,Active,0,0
298023,2675188,350201,-17,24.0,22.0,Active,0,0
895467,2675188,350201,-16,24.0,21.0,Active,0,0
3780414,2675188,350201,-15,24.0,20.0,Active,0,0
4238615,2675188,350201,-14,24.0,19.0,Active,0,0
3778202,2675188,350201,-13,24.0,18.0,Active,0,0
2101807,2675188,350201,-12,24.0,17.0,Active,0,0
675399,2675188,350201,-11,24.0,16.0,Active,0,0
1401746,2675188,350201,-10,24.0,15.0,Active,0,0


In [28]:
pos_features = pos

# POS_IS_ACTIVE_FLAG - flag for if they have any active credit - need to look at latest month only
pos_features = pos_features.assign(LATEST_MONTH=pos_features.groupby('SK_ID_PREV')['MONTHS_BALANCE'].transform('max'))
pos_features.loc[:, 'POS_IS_ACTIVE_FLAG'] = None
pos_features.loc[pos_features['MONTHS_BALANCE'] == pos_features['LATEST_MONTH'], 'POS_IS_ACTIVE_FLAG'] = 0
pos_features.loc[(pos_features['MONTHS_BALANCE'] == pos_features['LATEST_MONTH'])&(pos_features['NAME_CONTRACT_STATUS'] == 'Active'), 'POS_IS_ACTIVE_FLAG'] = 1

# sum total DPD in the last 1/6/12/24 months
# sum(SK_DPD) over SK_ID_CURR where months balance < X
pos_features = pos_features.assign(POS_DPD_LAST_1_MONTH=pos_features.loc[pos_features['MONTHS_BALANCE'] >= -1].groupby('SK_ID_CURR')['SK_DPD'].transform('sum'))
pos_features = pos_features.assign(POS_DPD_LAST_6_MONTHS=pos_features.loc[pos_features['MONTHS_BALANCE'] >= -6].groupby('SK_ID_CURR')['SK_DPD'].transform('sum'))
pos_features = pos_features.assign(POS_DPD_LAST_12_MONTHS=pos_features.loc[pos_features['MONTHS_BALANCE'] >= -12].groupby('SK_ID_CURR')['SK_DPD'].transform('sum'))
pos_features = pos_features.assign(POS_DPD_LAST_24_MONTHS=pos_features.loc[pos_features['MONTHS_BALANCE'] >= -24].groupby('SK_ID_CURR')['SK_DPD'].transform('sum'))

# POS_DPD_MAX - maximum days overdue
# POS_DPD_AVG - average days overdue
# POS_IS_ACTIVE_FLAG - flag if any credit is currently active
# POS_ACTIVE_SUM - number of lines of credit that are currently active
pos_features = pos_features.groupby(['SK_ID_CURR']).agg(
        POS_DPD_MAX=('SK_DPD', np.max), 
        POS_DPD_AVG=('SK_DPD', np.mean), 
        POS_IS_ACTIVE_FLAG=('POS_IS_ACTIVE_FLAG', np.max), 
        POS_ACTIVE_SUM=('POS_IS_ACTIVE_FLAG', np.sum), 
        POS_DPD_LAST_1_MONTH=('POS_DPD_LAST_1_MONTH', np.max),
        POS_DPD_LAST_6_MONTHS=('POS_DPD_LAST_6_MONTHS', np.max), 
        POS_DPD_LAST_12_MONTHS=('POS_DPD_LAST_12_MONTHS', np.max), 
        POS_DPD_LAST_24_MONTHS=('POS_DPD_LAST_24_MONTHS', np.max)
        )


pos_features
# pos_features[pos_features['SK_ID_CURR'] == 100201].drop_duplicates()

,POS_DPD_MAX,POS_DPD_AVG,POS_IS_ACTIVE_FLAG,POS_ACTIVE_SUM,POS_DPD_LAST_1_MONTH,POS_DPD_LAST_6_MONTHS,POS_DPD_LAST_12_MONTHS,POS_DPD_LAST_24_MONTHS
SK_ID_CURR,,,,,,,,
100002,0,0.000000,1,1,0.0,0.0,0.0,0.0
100003,0,0.000000,1,1,NaN,NaN,NaN,0.0
100004,0,0.000000,0,0,NaN,NaN,NaN,0.0
100006,0,0.000000,1,1,0.0,0.0,0.0,0.0
100007,0,0.000000,1,2,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
456251,0,0.000000,0,0,0.0,0.0,0.0,0.0
456252,0,0.000000,0,0,NaN,NaN,NaN,NaN
456253,5,0.294118,1,1,NaN,NaN,NaN,NaN


In [29]:
# pos_features.loc[pos_features['SK_ID_PREV']==2675188].sort_values('MONTHS_BALANCE')
pos_features.filter(items=[350201], axis=0)

,POS_DPD_MAX,POS_DPD_AVG,POS_IS_ACTIVE_FLAG,POS_ACTIVE_SUM,POS_DPD_LAST_1_MONTH,POS_DPD_LAST_6_MONTHS,POS_DPD_LAST_12_MONTHS,POS_DPD_LAST_24_MONTHS
SK_ID_CURR,,,,,,,,
350201,46,1.142857,1,1,15.0,104.0,104.0,104.0


In [30]:
pos_features.to_pickle(data_output_dir + "pos_features.pkl")

In [44]:
# check the proportion defaults for loans with DPD>0
test = pos_features.merge(home_loan_train[['TARGET']], on='SK_ID_CURR')
test.loc[:, 'POS_DPD_FLAG'] = '0'
test.loc[test['POS_DPD_MAX'] > 0, 'POS_DPD_FLAG'] = '1'
test[['TARGET', 'POS_DPD_FLAG']].groupby('POS_DPD_FLAG').mean()

,TARGET
POS_DPD_FLAG,
0,0.077600
1,0.099638
